the task is to tune the RF model for each spot. there will be one set of hyperparameters used for the whole thing.
- i set spot 09 to be used as the test data. the accuracy and other necessary metrics will be determined from there.
- 


In [83]:
import pandas as pd
path = ''
df_01 = pd.read_csv(path+"df_01.csv")
df_02 = pd.read_csv(path+"df_02.csv")
df_03 = pd.read_csv(path+"df_03.csv")
df_04 = pd.read_csv(path+"df_04.csv")
df_05 = pd.read_csv(path+"df_05.csv")
df_06 = pd.read_csv(path+"df_06.csv")
df_07 = pd.read_csv(path+"df_07.csv")
df_09 = pd.read_csv(path+"df_09.csv")


In [84]:
merge = pd.concat([df_01,df_02,df_03,df_04,df_05,df_06,df_07,df_09]);merge=merge.dropna(subset=['laf50_per_hour']) #drop NAs in the 'laf50_per_hour'
#keep in mind that operations done on train do not apply for the constituent dataframes.
#convert 'date' and 'day' to datetime for both train and test
merge['date'] = pd.to_datetime(merge['date']); merge['day'] = pd.to_datetime(merge['date'])

weekday_mapping = {
    'Monday': 1,
    'Tuesday': 2,
    'Wednesday': 3,
    'Thursday': 4,
    'Friday': 5,
    'Saturday': 6,
    'Sunday': 7
}
merge['weekday'] = merge['weekday'].map(weekday_mapping)

In [85]:
merge.head(1)

,date,day,time,start_hour,weekday,LC_HUMIDITY,LC_RAININ,LC_TEMP_QCL3_list,school,loko,depot,location,Latitude,Longitude,laf50_per_hour,baseline,noisiness
157,2022-03-07 13:00:00,2022-03-07 13:00:00,13:00-14:00,13,1,42.873016,0.0,5.789435,0.0,1.0,2.0,Number 35,50.877146,4.700688,51.1,53.243815,False


i want to try using df_09 as validation, then tune hyperparameters with sklearn's GridSearchCV(). 
next i'll try to see if i can use nested CV: 7-fold hyperparameter tuning, in a sense.

In [86]:
predictor_vars = ['start_hour','weekday','LC_HUMIDITY','LC_RAININ','LC_TEMP_QCL3_list','school','loko','depot']
merge_X = merge.loc[:,predictor_vars]
merge_Y = merge.loc[:,'noisiness']

In [90]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
X_tv, X_test, Y_tv, Y_test = train_test_split(merge_X, merge_Y, test_size=0.1, random_state=19)
X_train, X_valid, Y_train, Y_valid = train_test_split(X_tv,Y_tv,test_size=0.3,random_state=19)
model = RandomForestClassifier()
model.fit(X_train,Y_train)


RandomForestClassifier()

In [89]:
print(len(X_tv))
print(len(X_test))

44109
4901


In [91]:
from sklearn.metrics import accuracy_score
#prediction checking
Y_pred = model.predict(X_valid)
accuracy = accuracy_score(Y_valid,Y_pred)
print(accuracy)

0.8332955490062722


83.3% is only done on validation data. 

In [92]:
#what if we tested on df_09 as planned? does it generalize?
Y_test_pred = model.predict(X_test)
accuracy_test = accuracy_score(Y_test_pred,Y_test)
print(accuracy_test)

0.8306468067741277


it generalizes a teensy bit better. now to try tuning, with RandomizedSearchCV() apparently being used before GridSearchCV().
source: https://www.geeksforgeeks.org/random-forest-hyperparameter-tuning-in-python/

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
model = RandomForestClassifier()
param_grid_rs = {
    'criterion': ['entropy'],
    'max_depth': [20, 30, 40, 50],
    'min_samples_leaf': [4, 10, 50, 100],
    'min_samples_split': [2, 10, 20, 50],
    'max_leaf_nodes': [2,4,6,8,10],
    'n_estimators': [200, 400, 600, 800, 1000],
    'max_samples': [0.2,0.4,0.6]
}
rf_random = RandomizedSearchCV(estimator = model, param_distributions = param_grid_rs, cv = 4, verbose=2, random_state=19)
rf_random.fit(X_train, Y_train) #fit is to initiate training process


In [95]:
print(rf_random.best_estimator_)

RandomForestClassifier(criterion='entropy', max_depth=20, max_leaf_nodes=10,
                       max_samples=0.4, min_samples_leaf=4, n_estimators=800)


thus GridSearchCV() will be trained around:

criterion='entropy', max_depth=20, max_leaf_nodes=10,
                       max_samples=0.4, min_samples_leaf=4, n_estimators=800

In [ ]:
from sklearn.model_selection import GridSearchCV
param_grid_gs = {
    'criterion': ['entropy'],
    'max_depth': [15,20,25],
    'min_samples_leaf': [2,4,6],
    'max_leaf_nodes': [9,10,11],
    'n_estimators': [800],
    'max_samples': [0.4]
}
rf_grid = GridSearchCV(estimator = model, param_grid = param_grid_gs, cv = 3, verbose=2)
rf_grid.fit(X_train, Y_train) #fit is to initiate training process

In [98]:
print(rf_grid.best_estimator_)

RandomForestClassifier(criterion='entropy', max_depth=25, max_leaf_nodes=11,
                       max_samples=0.4, min_samples_leaf=6, n_estimators=800)


criterion='entropy', max_depth=25, max_leaf_nodes=11,
                       max_samples=0.4, min_samples_leaf=6, n_estimators=800

In [100]:
#test data accuracy?
from sklearn.metrics import accuracy_score
param_grid_final = {
    'criterion': ['entropy'],
    'max_depth': [25],
    'min_samples_leaf': [6],
    'max_leaf_nodes': [11],
    'n_estimators': [800],
    'max_samples': [0.4]
}
model.fit(X_tv,Y_tv)
Y_test_pred = model.predict(X_test)
accuracy = accuracy_score(Y_test,Y_test_pred)
print(accuracy)

0.8453376861864925


marginal gains. seems ok? time to train on each location and see.

In [110]:
# Create an empty dictionary to store accuracy scores
accuracy_scores = {}
# Define the mapping of weekdays to numerical values
weekday_mapping = {
    'Monday': 1,
    'Tuesday': 2,
    'Wednesday': 3,
    'Thursday': 4,
    'Friday': 5,
    'Saturday': 6,
    'Sunday': 7
}
predictor_vars = ['start_hour','weekday','LC_HUMIDITY','LC_RAININ','LC_TEMP_QCL3_list','school','loko','depot']
data = [df_01,df_02,df_03,df_04,df_05,df_06,df_07,df_09]
name_df = ['df_01','df_02','df_03','df_04','df_05','df_06','df_07','df_09']
param_grid_final = {
    'criterion': ['entropy'],
    'max_depth': [25],
    'min_samples_leaf': [6],
    'max_leaf_nodes': [11],
    'n_estimators': [800],
    'max_samples': [0.4]
}
iter_c = 0
for iter in data:
  data_it = pd.DataFrame(iter)
  data_it = data_it.dropna(subset=['laf50_per_hour'])
  data_it['weekday'] = data_it['weekday'].map(weekday_mapping)
  data_it['date'] = pd.to_datetime(data_it['date'])
  X = data_it.loc[:,predictor_vars]
  Y = data_it.loc[:,'noisiness']

  X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=19)
  rf_model = RandomForestClassifier(criterion='entropy',max_depth=25,min_samples_leaf=6,
                                    max_leaf_nodes=6, n_estimators=800,max_samples=0.4,
                                    random_state=19)
  rf_model.fit(X_train, y_train)
  y_pred_rf = rf_model.predict(X_test)
  accuracy_rf = accuracy_score(y_test, y_pred_rf)
  accuracy_scores[name_df[iter_c]] = {
        'Random Forest Accuracy': accuracy_rf}
  iter_c = iter_c+1
#loop ends
accuracy_df = pd.DataFrame.from_dict(accuracy_scores, orient='index')
print(accuracy_df)


<ipython-input-110-aa3233084e1d>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_it['weekday'] = data_it['weekday'].map(weekday_mapping)
<ipython-input-110-aa3233084e1d>:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_it['date'] = pd.to_datetime(data_it['date'])


       Random Forest Accuracy
df_01                0.745982
df_02                0.774935
df_03                0.799365
df_04                0.814545
df_05                0.790286
df_06                0.773945
df_07                0.783799
df_09                0.673415
